In [1]:
# Standard modules
import glob
import os
import shutil

# Willow Fox Fortino's modules
import GPRutils
import vK2KGPR
import plotGPR

# Professor Gary Bernstein's modules
import getGaiaDR2 as gaia
import gbutil
import vonkarmanFT as vk

# Science modules
import numba
import numpy as np
import astropy.units as u
import astropy.constants as c
import astropy.coordinates as co
import astropy.table as tb
import astropy.io.fits as fits
import astropy.stats as stats
from astropy.time import Time
import scipy.optimize as opt
from scipy.spatial.ckdtree import cKDTree

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


Numba doesn't seem to help with functions fit and predict. Maybe inside vonkarmanFT.py

In [21]:
exposure = 348819
dC = GPRutils.loadFITS(glob.glob(f"../thesis/{exposure}/DES{exposure}_?.fits")[0])
dC2 = GPRutils.loadNPZ(f"../thesis/{exposure}/{exposure}.npz")
dC.params = dC2.params

In [22]:
Train, Valid = GPRutils.makeSplit(dC.TV, "Subset A")
dC.makeArrays(Train, Valid)

In [43]:
%%time
ttt = vk.TurbulentLayer(
    variance=dC.params[0],
    outerScale=dC.params[1],
    diameter=dC.params[2],
    wind=(dC.params[3], dC.params[4]))

CPU times: user 33.2 s, sys: 5.19 s, total: 38.4 s
Wall time: 11.4 s


In [55]:
%%time
du, dv = GPRutils.getGrid(dC.Xtrain, dC.Xtrain) 
CuvTT = ttt.getCuv(du, dv)
CuvTT[:, :, 0, 1] *= 0
CuvTT[:, :, 1, 0] *= 0
n1, n2 = CuvTT.shape[0], CuvTT.shape[1]
K = np.swapaxes(CuvTT, 1, 2).reshape(2*n1, 2*n2)

CPU times: user 15min 55s, sys: 2min 39s, total: 18min 35s
Wall time: 1min


In [56]:
%%time
du, dv = GPRutils.getGrid(dC.Xvalid, dC.Xtrain)
CuvVT = ttt.getCuv(du, dv)
CuvVT[:, :, 0, 1] *= 0
CuvVT[:, :, 1, 0] *= 0
n1, n2 = CuvVT.shape[0], CuvVT.shape[1]
Ks = np.swapaxes(CuvVT, 1, 2).reshape(2*n1, 2*n2)

CPU times: user 3min 56s, sys: 36.2 s, total: 4min 32s
Wall time: 15.7 s


In [60]:
%%time
W = GPRutils.makeW(dC.Etrain_GAIA, dC.Etrain_DES)

CPU times: user 5.73 s, sys: 5.39 s, total: 11.1 s
Wall time: 5.93 s


In [62]:
%%time
Y = GPRutils.flat(dC.Ytrain)

CPU times: user 1.1 ms, sys: 0 ns, total: 1.1 ms
Wall time: 587 µs


In [57]:
%%time
L = np.linalg.cholesky(K + W)
alpha = np.linalg.solve(L, Y)
alpha = np.linalg.solve(L.T, alpha)

fbar_s = GPRutils.unflat(np.dot(Ks.T, alpha))

CPU times: user 18min 53s, sys: 45 s, total: 19min 38s
Wall time: 1min 46s


In [68]:
@numba.njit("f8[:](f8[:,:], f8[:,:], f8[:,:], f8[:])", cache=True)
def func(K, Ks, W, Y):
    
    # Get alpha
    L = np.linalg.cholesky(K + W)
    alpha = np.linalg.solve(L, Y)
    alpha = np.linalg.solve(L.T, alpha)
    
    # Get fbar_s
    fbar_s = np.dot(Ks.T, alpha)

    return fbar_s

<ipython-input-68-babc20051f88>:10: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 1d, C))
  fbar_s = np.dot(Ks.T, alpha)


In [69]:
%%time
fbar_s_numba = func(K, Ks, W, Y)

CPU times: user 18min 42s, sys: 33.5 s, total: 19min 15s
Wall time: 1min 57s


In [70]:
%%time
fbar_s_numba = func(K, Ks, W, Y)

CPU times: user 18min 53s, sys: 35.9 s, total: 19min 29s
Wall time: 1min 58s


In [71]:
%%time
fbar_s_numba = func(K, Ks, W, Y)

CPU times: user 17min 17s, sys: 36 s, total: 17min 53s
Wall time: 1min 51s


In [73]:
%%time
fbar_s_numba = func(K, Ks, W, Y)

CPU times: user 14min 34s, sys: 27.7 s, total: 15min 2s
Wall time: 1min 33s


In [74]:
%%time
L = np.linalg.cholesky(K + W)
alpha = np.linalg.solve(L, Y)
alpha = np.linalg.solve(L.T, alpha)

fbar_s = GPRutils.unflat(np.dot(Ks.T, alpha))

CPU times: user 14min 34s, sys: 33 s, total: 15min 7s
Wall time: 1min 18s


In [75]:
@numba.njit("f8[:](f8[:,:], f8[:,:], f8[:,:], f8[:])", cache=True, parallel=True)
def pfunc(K, Ks, W, Y):
    
    # Get alpha
    L = np.linalg.cholesky(K + W)
    alpha = np.linalg.solve(L, Y)
    alpha = np.linalg.solve(L.T, alpha)
    
    # Get fbar_s
    fbar_s = np.dot(Ks.T, alpha)

    return fbar_s

<ipython-input-75-c5d9dcc50912>:10: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 1d, C))
  fbar_s = np.dot(Ks.T, alpha)


In [76]:
%%time
fbar_s_numba = pfunc(K, Ks, W, Y)

CPU times: user 14min 36s, sys: 36.9 s, total: 15min 13s
Wall time: 1min 30s


In [77]:
%%time
fbar_s_numba = pfunc(K, Ks, W, Y)

CPU times: user 14min 27s, sys: 40.3 s, total: 15min 7s
Wall time: 1min 30s
